In [9]:
import json
from enum import Enum
import numpy as np

In [196]:
from graph import  Node, Link, TYPES_OF_GRAPH

In [132]:
with open('./ground_truth_graph.json', 'r') as f:
    ground_truth = json.load(f)

with open('./generated_graph.json', 'r') as g:
    generated = json.load(g)

In [217]:
class Graph:
    def __init__(self, graph: dict, type: TYPES_OF_GRAPH) -> None:
        node_count = len(graph['nodes'])
        self.type = type
        self.nodes = []
        self.transitions = [[None for _ in range(node_count + 1)] for _ in range(node_count + 1)]
        self.graph_degrees = []
        for node in graph['nodes']:
            self.nodes.append(Node(node['id'], node['is_start'], node['label'], node['response']))
        for link in graph['links']:
            first = link['source']
            second = link['target']
            link_obj = Link(first, second)
            link_obj.add_response(link['request'])
            self.transitions[first][second] = link_obj
        for i in range(1, node_count + 1):
            self.graph_degrees.append(sum(1 for x in self.transitions[i] if x is not None))


In [218]:
ground_truth_graph = Graph(ground_truth, 1)
generated_graph = Graph(generated, 1)

In [219]:
ground_truth_graph.graph_degrees

[2, 1, 1, 2, 1, 1, 0]

In [213]:
generated_graph.graph_degrees

[2, 1, 1, 2, 1, 1, 0]

In [224]:
def check_node_identical(graph_1, graph_2):
       #check if we have the same amount of nodes:
    if len(graph_1.nodes) != len(graph_2.nodes):
        return False
    
    #check if the nodes are in the same 
    unmatched_1 = set(graph_1.nodes)
    unmatched_2 = set(graph_2.nodes)
    return unmatched_1 == unmatched_2


In [225]:
check_node_identical(generated_graph, ground_truth_graph)

True